<a href="https://colab.research.google.com/github/chethanabhaskara/Early-diagonsis-of-Alzheimer-s-disease/blob/master/DNN_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import tensorflow as tf
import numpy as np

from pandas import read_csv
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import recall_score, roc_curve, auc

# Preprocess input
'''
  Reads the longitudinal and cross_sectional datasets into pandas dataframe, handles
  missing values and converts categorical data into numbers.
'''
longitudinal=pd.read_csv('https://raw.githubusercontent.com/chethanabhaskara/Dimentia-Early-Diagnosis/master/oasis_longitudinal.csv')
longitudinal=longitudinal.fillna(method='ffill')
cross_sectional=pd.read_csv('https://raw.githubusercontent.com/chethanabhaskara/Dimentia-Early-Diagnosis/master/oasis_cross-sectional.csv')
cross_sectional=cross_sectional.fillna(method='ffill')
for x in longitudinal.columns:
    f = LabelEncoder()
    longitudinal[x] = f.fit_transform(longitudinal[x])
for x in cross_sectional.columns:
    f = LabelEncoder()
    cross_sectional[x] = f.fit_transform(cross_sectional[x])

# Concatenate datasets
''' 
  Combines the two datasets and introduces a new attribute called 'Group' based off of the 
  CDR attributes values for an easier classification and training of the datapoints.
'''
combined=pd.concat([longitudinal,cross_sectional])
group = combined['CDR']
group = group.apply(lambda cdr: 1 if cdr>=0.5 else 0)
combined['Group'] = group
combined.head()
combined=combined.fillna(method='ffill')

# Split the dataset into training and testing
train, test = train_test_split(combined, test_size=0.2, random_state=0)
X_train = train[['ASF', 'Age', 'EDUC', 'Hand', 'M/F','MMSE','MR Delay','SES','eTIV','nWBV']]
X_test = test[['ASF', 'Age', 'EDUC', 'Hand', 'M/F','MMSE','MR Delay','SES','eTIV','nWBV']]
Y_train = train.Group
Y_test = test.Group

# Scale the data set 
'''
   Transforms and scales the datasets so that it fits within a specific scale for each feature 
   attributes.
'''
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_train=np.asarray(X_train)
X_test=np.asarray(X_test)
Y_train=np.ravel(Y_train)
Y_test=np.ravel(Y_test)

# Define the classifier 
config = tf.estimator.RunConfig(tf_random_seed=0)
feature_columns = [tf.feature_column.numeric_column('x', shape=np.array(X_train).shape[1:])]
classifier = tf.estimator.DNNClassifier(feature_columns=feature_columns, hidden_units=[256,124,32], n_classes=4,
        weight_column= tf.feature_column.numeric_column('weight'),
        optimizer=lambda: tf.keras.optimizers.Adam(
        learning_rate=tf.compat.v1.train.exponential_decay(
            learning_rate=0.01,
            global_step=tf.compat.v1.train.get_global_step(),
            decay_steps=250,
            decay_rate=0.1)),
        config=config)

# Train the network
train_numpy_input_fn = tf.compat.v1.estimator.inputs.numpy_input_fn(x={'x': X_train, 'weight': np.ones(X_train.shape[0])}, y=Y_train, num_epochs=None, batch_size=20, shuffle=False)
classifier.train(input_fn=train_numpy_input_fn, steps=500)

# Predict output
test_numpy_input_fn = tf.compat.v1.estimator.inputs.numpy_input_fn(x={'x': X_test, 'weight': np.ones(X_test.shape[0])}, y=Y_test, num_epochs=1, shuffle=False)
predictions = classifier.predict(input_fn=test_numpy_input_fn)
y_predicted = np.array(list(i['class_ids'] for i in predictions))
y_predicted = y_predicted.reshape(np.array(Y_test).shape)

# Calculate score
scores = classifier.evaluate(input_fn=test_numpy_input_fn)
print('Accuracy: {0:f}'.format(scores['accuracy']))
print('Recall: {0:f}'.format(recall_score(Y_test, y_predicted, pos_label=1)))
fpr, tpr, thresholds = roc_curve(Y_test, y_predicted, pos_label=1)
print('AUC: {0:f}'.format(auc(fpr, tpr)))

INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpaxaio3aq', '_tf_random_seed': 0, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only 